In [36]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

SentenceBERT 모델 로드

In [37]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510443 -0.77338403  0.5927713  ...  0.579235    0.32683447
  -0.6508968 ]
 [-0.09361703 -0.18191501 -0.19230817 ... -0.03165794  0.30412498
  -0.26793626]]


In [39]:
df = pd.read_csv('computer.csv')

df.head()

,품명,가격,링크
0,삼성전자 삼성 DM500TDA-A38A,"741,040원",https://adcr.naver.com/adcr?x=lRY2h8N2wcCFgin2...
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원",https://adcr.naver.com/adcr?x=yR/6t66fjsTCfIaK...
2,한성컴퓨터 TFG MX9465S,"878,000원",https://adcr.naver.com/adcr?x=XG0vA8W5qu0g8MZO...
3,삼성전자 삼성 DM530ADA-L15A,"592,630원",https://cr.shopping.naver.com/adcr.nhn?x=kYYcN...
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원",https://cr.shopping.naver.com/adcr.nhn?x=MCgi3...


전처리

In [40]:
df = df.drop(columns=['링크'])

df.head()

,품명,가격
0,삼성전자 삼성 DM500TDA-A38A,"741,040원"
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원"
2,한성컴퓨터 TFG MX9465S,"878,000원"
3,삼성전자 삼성 DM530ADA-L15A,"592,630원"
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원"


In [41]:
df.loc[0, '품명']

'삼성전자 삼성 DM500TDA-A38A'

In [42]:
model.encode(df.loc[0, '품명'])

array([ 4.54589218e-01,  2.62408584e-01, -6.83216006e-02, -3.06488089e-02,
       -1.82356820e-01, -4.14297879e-01,  4.32798117e-01, -1.78386688e-01,
       -3.08389217e-01,  2.77258843e-01, -1.69698969e-01,  2.68526763e-01,
       -4.24424022e-01,  3.56342643e-01, -8.08301866e-02,  3.05231214e-01,
       -1.68940932e-01,  5.08443296e-01, -1.05127737e-01, -1.36436331e+00,
       -1.28231421e-01,  5.15901782e-02,  4.82359618e-01,  1.05233543e-01,
        1.95238844e-01,  3.16972554e-01, -1.64200410e-01, -2.88950384e-01,
        2.06479162e-01, -2.11312380e-02,  1.43728897e-01, -3.24560374e-01,
        1.41150370e-01, -6.94961429e-01,  1.52256191e-01, -4.46219862e-01,
       -4.03074741e-01, -2.87984051e-02, -5.64190686e-01, -8.04996073e-01,
        2.07443744e-01, -3.39599070e-03,  2.83288777e-01,  3.35777968e-01,
        8.02419245e-01, -2.34468773e-01,  2.73429662e-01,  2.20359340e-01,
        3.80189717e-01, -2.30992898e-01, -2.10753381e-01,  1.01641759e-01,
       -2.75166165e-02, -

유저 대화내용 인코딩

In [43]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['품명'].map(lambda x: list(model.encode(x)))

df.head()

,품명,가격,embedding
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648..."
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741..."
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424..."
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,..."
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239..."


In [44]:
df.to_csv('computer1.csv', index=False)

간단한 챗봇

In [45]:
text = '삼성전자 삼성 DM530ADA-L78A'

embedding = model.encode(text)

In [46]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.850676
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.865888
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.462458
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.876439
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",1.000000


In [47]:
answer = df.loc[df['similarity'].idxmax()] # 최대값 뽑기

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 1,333,660원
유사도 1.0


In [48]:
text = 'IGNIQ 비와이 30Y'

embedding = model.encode(text)

In [49]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.278538
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.172590
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.311813
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.211781
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.181291


In [50]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 1,099,000원
유사도 1.0000001192092896


In [51]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.278538
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.172590
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.311813
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.211781
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.181291
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041149, 0.347112, 0.4194436, 0.23224746,...",0.318181
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.1671801, -0.06351817, 0.108958244, 0.24156...",0.185421
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.1545794, -0.040598195, -0.25815138, -0.4596...",0.116212
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.3670941, 0.06672245, 0.014325255, 0.363603...",0.249845


In [52]:
text = '삼성전자 삼성 DM500SDZ-AD5A'

embedding = model.encode(text)

In [53]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.834929
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",1.000000
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.497394
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.895725
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.865888


In [54]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 729,000원
유사도 1.0


In [ ]:
text = '사무용 컴퓨터 데스크탑 10세대'

embedding = model.encode(text)

In [55]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.834929
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",1.000000
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.497394
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.895725
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.865888


In [ ]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 329,000원
유사도 0.7316416501998901


In [56]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.834929
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",1.000000
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.497394
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.895725
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.865888
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041149, 0.347112, 0.4194436, 0.23224746,...",0.336201
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.1671801, -0.06351817, 0.108958244, 0.24156...",0.616914
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.1545794, -0.040598195, -0.25815138, -0.4596...",0.362092
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.3670941, 0.06672245, 0.014325255, 0.363603...",0.290882


In [57]:
text = ' 사무용컴퓨터'

embedding = model.encode(text)

In [58]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.320978
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.305800
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.563971
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.312021
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.305867


In [59]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 805,000원
유사도 0.7380939722061157


In [ ]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM530ADA-L78A,"1,348,000원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.305867
1,삼성전자 삼성 DM500TDA-A38A,"749,000원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.320978
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.563971
3,삼성전자 삼성 DM530ADA-L78A,"1,348,000원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.305867
4,삼성전자 삼성 DM530ADA-L15A,"597,000원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.312021
5,삼성전자 삼성 DM530ADA-L58A,"1,148,000원","[0.39266935, 0.38629276, 0.35971734, -0.100096...",0.274878
6,한성컴퓨터 프리워커 F2500W,"588,000원","[-0.28878358, 0.3785906, 0.3887592, -0.0283242...",0.503836
7,삼성전자 삼성 DM500TDA-A58A,"898,000원","[0.42466667, 0.31970465, 0.10830757, -0.028160...",0.264236
8,IGNIQ 리트 올인원 PC 27A,"469,000원","[0.5777558, 0.21352364, -0.092224754, 0.459173...",0.454530
9,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.563971


In [60]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.320978
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.305800
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.563971
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.312021
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.305867
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041149, 0.347112, 0.4194436, 0.23224746,...",0.432733
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.1671801, -0.06351817, 0.108958244, 0.24156...",0.650922
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.1545794, -0.040598195, -0.25815138, -0.4596...",0.614997
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.3670941, 0.06672245, 0.014325255, 0.363603...",0.358972


In [61]:
text = ' A24-310 '

embedding = model.encode(text)

In [62]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.472704
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.413657
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.427432
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.382098
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.396787


In [63]:
answer = df.loc[df['similarity'].idxmax()]

print('챗봇 답변', answer['가격'])
print('유사도', answer['similarity'])

챗봇 답변 699,000원
유사도 0.6007322072982788


In [64]:
df

,품명,가격,embedding,similarity
0,삼성전자 삼성 DM500TDA-A38A,"741,040원","[0.45458922, 0.26240858, -0.0683216, -0.030648...",0.472704
1,삼성전자 삼성 DM500SDZ-AD5A,"729,000원","[0.4549086, 0.34968653, 0.22412099, -0.3557741...",0.413657
2,한성컴퓨터 TFG MX9465S,"878,000원","[-0.4821301, 0.023954194, 0.024152594, -0.4424...",0.427432
3,삼성전자 삼성 DM530ADA-L15A,"592,630원","[0.43844014, 0.3671505, 0.5040723, -0.5082711,...",0.382098
4,삼성전자 삼성 DM530ADA-L78A,"1,333,660원","[0.37922522, 0.44932583, 0.28506005, -0.277239...",0.396787
...,...,...,...,...
822,디클 A24-310 윈도우,"699,000원","[-0.21041149, 0.347112, 0.4194436, 0.23224746,...",0.600732
823,삼성데스크탑본체 i5 8GB SSD512GB 정품윈도우11 삼성컴퓨터본체 사무용컴퓨터,"1,018,890원","[-0.1671801, -0.06351817, 0.108958244, 0.24156...",0.387941
824,인텔 11세대 i5-11400/8G/256G/컴집조립PC/가정.사무용,"397,440원","[0.1545794, -0.040598195, -0.25815138, -0.4596...",0.227867
825,Apple 아이맥 24형 2021년형 M1 8코어 CPU 및 8코어 GPU 256G...,"1,806,140원","[-0.3670941, 0.06672245, 0.014325255, 0.363603...",0.367418
